In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Set working directory 
import os
os.chdir('/content/gdrive/My Drive')

In [3]:
!pip install pykml
# Importing packages
import requests
from pykml import parser

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Data of resale flats and their prices
df = pd.read_csv("resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
print(len(df))
df.head()

136173


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [5]:
# Getting the unique street names of flats to retreive their coordinates
df1 = df.groupby("street_name").count().reset_index()
print(len(df1))
df1.head()

559


,street_name,month,town,flat_type,block,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,ADMIRALTY DR,543,543,543,543,543,543,543,543,543,543
1,ADMIRALTY LINK,414,414,414,414,414,414,414,414,414,414
2,AH HOOD RD,29,29,29,29,29,29,29,29,29,29
3,ALJUNIED AVE 2,1,1,1,1,1,1,1,1,1,1
4,ALJUNIED CRES,238,238,238,238,238,238,238,238,238,238


In [6]:
# List of unique street names
street_list = list(df1['street_name'])
street_list[:5]

['ADMIRALTY DR',
 'ADMIRALTY LINK',
 'AH HOOD RD',
 'ALJUNIED AVE 2',
 'ALJUNIED CRES']

In [7]:
df["address"] = df['block'] + " " + df['street_name']
df1 = df.groupby("address").count().reset_index()
print(len(df1))
street_list = list(df1['address'])

9317


In [8]:
# Send request to OneMap to retrieve coordinates of streets
def getcoordinates(address):
    req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    resultsdict = eval(req.text)
    if len(resultsdict['results'])>0:
        return resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE']
    else:
        pass

In [9]:
# Retrieve coordinates (will take some time)
coordinates_list= []
count = 0
failed_count = 0
for address in street_list:
    try:
        if len(getcoordinates(address))>0:
            count = count + 1
            print('Extracting',count,'out of',len(street_list),'addresses')
            coordinates_list.append(getcoordinates(address))
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len(street_list),'addresses')
        coordinates_list.append(None)
print('Total Number of Addresses With No Coordinates',failed_count)

Streaming output truncated to the last 5000 lines.
Extracting 4319 out of 9317 addresses
Extracting 4320 out of 9317 addresses
Extracting 4321 out of 9317 addresses
Extracting 4322 out of 9317 addresses
Extracting 4323 out of 9317 addresses
Extracting 4324 out of 9317 addresses
Extracting 4325 out of 9317 addresses
Extracting 4326 out of 9317 addresses
Extracting 4327 out of 9317 addresses
Extracting 4328 out of 9317 addresses
Extracting 4329 out of 9317 addresses
Extracting 4330 out of 9317 addresses
Extracting 4331 out of 9317 addresses
Extracting 4332 out of 9317 addresses
Extracting 4333 out of 9317 addresses
Extracting 4334 out of 9317 addresses
Extracting 4335 out of 9317 addresses
Extracting 4336 out of 9317 addresses
Extracting 4337 out of 9317 addresses
Extracting 4338 out of 9317 addresses
Extracting 4339 out of 9317 addresses
Extracting 4340 out of 9317 addresses
Extracting 4341 out of 9317 addresses
Extracting 4342 out of 9317 addresses
Extracting 4343 out of 9317 addresses

In [10]:
# Join the dataframe of coordinates with dataframe of unique street names
df_coordinates = pd.DataFrame(coordinates_list)
df1 = df1.join(df_coordinates)
df1 = df1.rename(columns={0:'Latitude', 1:'Longitude'}) 
#df1 = df1[["street_name", "Latitude", "Longitude"]]
df1 = df1[["address", "Latitude", "Longitude"]]
print(len(df1))
df1.head()

9317


,address,Latitude,Longitude
0,1 BEACH RD,1.3036713506088,103.864478660925
1,1 BEDOK STH AVE 1,1.32085208689731,103.933721091441
2,1 CHAI CHEE RD,1.32796879176302,103.922716018139
3,1 CHANGI VILLAGE RD,1.38854656229022,103.987804503483
4,1 DELTA AVE,1.2920752508431,103.828584077626


In [11]:
# Left join the dataframe of unique street names and their coordinates with the flat sales data on street name
#df_combined = pd.merge(df, df1, left_on='street_name', right_on='street_name')
df_combined = pd.merge(df, df1, left_on='address', right_on='address')
print(len(df_combined))
df_combined.head()

136173


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,Latitude,Longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,1.36200453938712,103.853879910407
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 01 month,235000.0,406 ANG MO KIO AVE 10,1.36200453938712,103.853879910407
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,202000.0,406 ANG MO KIO AVE 10,1.36200453938712,103.853879910407
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,210000.0,406 ANG MO KIO AVE 10,1.36200453938712,103.853879910407
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,60 years 01 month,220000.0,406 ANG MO KIO AVE 10,1.36200453938712,103.853879910407


In [12]:
# Parse kml file to get data of expressways
kml_file = "national-map-line-kml.kml"
with open(kml_file) as f:
    doc = parser.parse(f)

In [13]:
# Loading data into list 
name, folder_path, symbol_id, inc, fmel = [], [], [], [], []
coord = []

for tags in doc.getroot().Document.Folder.Placemark:      
    for tmp in tags.ExtendedData.SchemaData.SimpleData:
        if tmp.attrib.get('name') == "NAME":
            name.append(tmp.text)
        elif tmp.attrib.get('name') == "FOLDERPATH":
            folder_path.append(tmp.text)
        elif tmp.attrib.get('name') == "SYMBOLID":
            symbol_id.append(tmp.text)
        elif tmp.attrib.get('name') == "INC_CRC":
            inc.append(tmp.text)
        elif tmp.attrib.get('name') == "FMEL_UPD_D":
            fmel.append(tmp.text)

    for tmp in tags.LineString.coordinates:
        coord.append(tmp.text.split(" "))

In [14]:
# Converts kml file into pandas dataframe
df2 = pd.DataFrame(list(zip(name, folder_path, symbol_id, inc, fmel, coord)), columns=["Name", "FolderPath", "SymbolId", "Inc", "Fmel", "Coordinates"])
print(len(df2))
df2.head()

5134


,Name,FolderPath,SymbolId,Inc,Fmel,Coordinates
0,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,0C08DFFA475DDCCD,20191008154530,"[103.858333937416,1.3559533317473,0.0, 103.858..."
1,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,48A90A617CC124B8,20191008154530,"[103.857586091965,1.35610979081088,0.0, 103.85..."
2,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,051AA478B6209021,20191008154530,"[103.860424415728,1.36816548664505,0.0, 103.86..."
3,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,1C51FD53E1662A6B,20191008154530,"[103.85978033742,1.37228389603645,0.0, 103.860..."
4,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,44D0FFDF1EF47027,20191008154530,"[103.859369200799,1.36913452283533,0.0, 103.85..."


In [15]:
# Only use data that shows an expressway (we only need to the name and coordinates)
df3 = df2.loc[(df2["FolderPath"] == "Layers/Expressway_Sliproad")	| (df2["FolderPath"] == "Layers/Expressway")]
# We only need to the name and coordinates
df3 = df3[["Name", "Coordinates"]]
# Split the list of coordinates in each row to multiple rows of Latitude and Longitude
df3 = pd.DataFrame([[p, t] for p, T in df3.values for t in T], columns=df3.columns)
coord = df3["Coordinates"].str.split(",", expand = True)
df3["Longitude"] = coord[0]
df3["Latitude"] = coord[1]
df3.drop(columns =["Coordinates"], inplace = True)
print(len(df3))
df3.head()

7836


,Name,Longitude,Latitude
0,CENTRAL EXPRESSWAY,103.858333937416,1.3559533317473
1,CENTRAL EXPRESSWAY,103.858215578815,1.355816304599
2,CENTRAL EXPRESSWAY,103.858116866331,1.35575566979974
3,CENTRAL EXPRESSWAY,103.857992826192,1.35571405765487
4,CENTRAL EXPRESSWAY,103.85787572257,1.35572105501446


In [16]:
# Chaning data types to float and rounding off the coordinates
# NOTE: we are treating rounding off of coordinates as an approximation for how close the expressway is from the house
df3["Longitude"] = df3["Longitude"].astype("float64").apply(lambda x: round(x, 3))
df3["Latitude"] = df3["Latitude"].astype("float64").apply(lambda x: round(x, 3))

df_combined["Longitude"] = df_combined["Longitude"].astype("float64").apply(lambda x: round(x, 3))
df_combined["Latitude"] = df_combined["Latitude"].astype("float64").apply(lambda x: round(x, 3))

In [17]:
# Left join the expressways with the flats data on Latitude and Longitude
# This will give NaN for flats that are not close to expressways and the expressway's name otherwise
df3.drop_duplicates(subset=['Latitude', "Longitude"], inplace=True)
df4 = pd.merge(df_combined, df3, left_on=['Latitude', "Longitude"], right_on=['Latitude', "Longitude"], how='left')
print(len(df4))
df4.head()

136173


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,Latitude,Longitude,Name
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,1.362,103.854,NaN
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 01 month,235000.0,406 ANG MO KIO AVE 10,1.362,103.854,NaN
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,202000.0,406 ANG MO KIO AVE 10,1.362,103.854,NaN
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,60 years 02 months,210000.0,406 ANG MO KIO AVE 10,1.362,103.854,NaN
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,07 TO 09,44.0,Improved,1979,60 years 01 month,220000.0,406 ANG MO KIO AVE 10,1.362,103.854,NaN


In [19]:
# Drop columns that are not needed in understanding how resale price are affected
df4.drop(["block", "street_name", "lease_commence_date", "Latitude", "Longitude", "address"], inplace=True, axis=1)
# Only keep the years in the remaining lease and transaction date
df4["remaining_lease"] = df4["remaining_lease"].apply(lambda x: int(x.split(" ")[0]))
df4["month"] = df4["month"].apply(lambda x: int(x.split("-")[0])).astype("object")
# Output 1 when the flat is close to an expressway else 0
df4["Name"] = df4["Name"].apply(lambda x: 0 if pd.isna(x) else 1)
print(len(df4))
df4.head()

136173


,month,town,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,resale_price,Name
0,2017,ANG MO KIO,2 ROOM,10 TO 12,44.0,Improved,61,232000.0,0
1,2017,ANG MO KIO,2 ROOM,10 TO 12,44.0,Improved,61,235000.0,0
2,2018,ANG MO KIO,2 ROOM,01 TO 03,44.0,Improved,60,202000.0,0
3,2018,ANG MO KIO,2 ROOM,01 TO 03,44.0,Improved,60,210000.0,0
4,2018,ANG MO KIO,2 ROOM,07 TO 09,44.0,Improved,60,220000.0,0


In [20]:
# Label encode the categorical variables
for i in df4.columns:
    if df4[i].dtype == "object" or df4[i].dtype == "bool":
        le = LabelEncoder()
        le.fit(df4[i])
        df4[i] = le.transform(df4[i])
print(len(df4))
df4.head()

136173


,month,town,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,resale_price,Name
0,0,0,1,3,44.0,5,61,232000.0,0
1,0,0,1,3,44.0,5,61,235000.0,0
2,1,0,1,0,44.0,5,60,202000.0,0
3,1,0,1,0,44.0,5,60,210000.0,0
4,1,0,1,2,44.0,5,60,220000.0,0


In [22]:
# Export cleaned dataset to be used in R for modelling
df4.to_csv("cleaned1.csv")